In [1]:
import boto3
import os
import json
from datetime import datetime
from datetime import timedelta, timezone
from jinja2 import Template

from aws_log_extractor import LogManager, LogFilter

from mylogger import CustomLogger
logger = CustomLogger()

###############################################################################
###############################################################################

REGION = os.environ["AWS_DEFAULT_REGION"]
lfn_client = boto3.client("lambda", region_name=REGION)
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")
logs_client = boto3.client("logs")
events_client = boto3.client("events")
iam_client = boto3.client("iam")
ses_client = boto3.client("ses", region_name=REGION)  # Choose your AWS region


#### SES Operations

In [3]:
emails = ["AMominNJ@gmail.com", "bbcredcap3@gmail.com", "A.Momin.NYC@gmail.com"]
# Create an email identity
[ses_client.verify_email_identity(EmailAddress=email) for email in emails]


[{'ResponseMetadata': {'RequestId': '37763a12-100d-4f73-aaac-4f93cfee843d',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Sun, 24 Aug 2025 14:13:44 GMT',
    'content-type': 'text/xml',
    'content-length': '248',
    'connection': 'keep-alive',
    'x-amzn-requestid': '37763a12-100d-4f73-aaac-4f93cfee843d'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': '7c60e4f6-42bf-4075-9fbd-f373ea2f010b',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Sun, 24 Aug 2025 14:13:44 GMT',
    'content-type': 'text/xml',
    'content-length': '248',
    'connection': 'keep-alive',
    'x-amzn-requestid': '7c60e4f6-42bf-4075-9fbd-f373ea2f010b'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': '7c27cb4a-fc60-440b-b99e-c6d4441fa49d',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Sun, 24 Aug 2025 14:13:44 GMT',
    'content-type': 'text/xml',
    'content-length': '248',
    'connection': 'keep-alive',
    'x-amzn-requestid': '7c27cb4a-fc60-440b-b99e-c6d444

-   **Why SES Times Out Inside VPC Lambda**:

    -   By default, a Lambda without a VPC has outbound internet access.
    -   Once you attach it to a VPC, Lambda runs inside ENIs in your private subnets.
    -   Private subnets don’t have internet access unless routed through a NAT Gateway.
    -   Since SES is a public AWS service (no VPC endpoint), the Lambda must go out to the internet.
    -   Without NAT, the call hangs until Lambda’s timeout → that’s what you’re seeing.

-   **For Lambda in VPC, AWS actually requires**:

    -   Lambda ENIs to be placed in private subnets
    -   Private subnets to route 0.0.0.0/0 → NAT Gateway (not directly to IGW)
    -   The NAT Gateway itself to sit in a public subnet (IGW-facing)

-   **So, you’ll need to split your subnets**:

    -   1 (or more) public subnet(s) for NAT Gateway (with route → IGW)
    -   1 (or more) private subnet(s) for Lambda (with route → NAT GW)

#### S3 Operations

In [5]:
TEMPLATE_S3_BUCKET = "gpc-cuckoo-bucket-htech"
TEMPLATE = "email-templates"

[
    s3_client.upload_file(
        f"./templates/{item}", TEMPLATE_S3_BUCKET, f"{TEMPLATE}/{item}"
    )
    for item in os.listdir("./templates") if item.endswith(".html")
]

[None, None, None]

In [ ]:
# key = "AWS-EFS-Data/aws_efs_testing.txt"
# s3_file = s3_client.get_object(Bucket=TEMPLATE_S3_BUCKET, Key=key)

In [51]:
def get_template_from_s3(key):
    """Loads and returns html template from Amazon S3"""
    s3 = boto3.client("s3")
    s3_file = s3.get_object(Bucket=TEMPLATE_S3_BUCKET, Key=key)

    try:
        template = Template(s3_file["Body"].read().decode("utf-8"))
    except Exception as e:
        logger.eror("Failed to load template")
        raise e
    else:
        logger.info(f"Template ({key}) has been obtained")

    return template

def render_come_to_work_template(employee_first_name):
    template = get_template_from_s3("email-templates/come_to_work.html")
    html_email = template.render(first_name=employee_first_name)
    plaintext_email = "Hello {0}, \nPlease remember to be into work by 8am".format(
        employee_first_name
    )
    return html_email, plaintext_email



In [52]:
EMPLOYEES = [
    {
        # You'll need to verify this email
        "email": "A.Momin.NYC@gmail.com",
        "first_name": "Homer",
        "last_name": "Simpson",
    },
]
html_email, plaintext_email = render_come_to_work_template(EMPLOYEES[0]["first_name"])


INFO: 2025-08-22 19:51:16 [1903018041.py:12] Template (email-templates/come_to_work.html) has been obtained


#### Invoke Lambda Function

In [74]:
payload1 = {"resources": ["come_to_work"]}
payload2 = {"resources": ["daily_tasks"]}
payload3 = {"resources": ["pickup"]}

LFN_NAME = "gpc_cuckoo"

response = lfn_client.invoke(
    FunctionName=LFN_NAME,
    # InvocationType="RequestResponse",  # 'RequestResponse' for synchronous execution
    InvocationType="Event",  # 'Event' for Asynchronous execution
    Payload=json.dumps(payload3),
)

logger.info(response)

INFO: 2025-08-23 18:35:54 [1337497117.py:14] {
    "ResponseMetadata": {
        "RequestId": "bd7abb4c-af5e-4791-bc1c-f68cfa9c4f72",
        "HTTPStatusCode": 202,
        "HTTPHeaders": {
            "date": "Sat, 23 Aug 2025 23:35:54 GMT",
            "content-length": "0",
            "connection": "keep-alive",
            "x-amzn-requestid": "bd7abb4c-af5e-4791-bc1c-f68cfa9c4f72",
            "x-amzn-remapped-content-length": "0",
            "x-amzn-trace-id": "root=1-68aa505a-0bb3ba2e4cb07921292591d7;parent=3b88de4e85a1e041;sampled=0"
        },
        "RetryAttempts": 0
    },
    "StatusCode": 202,
    "Payload": "<botocore.response.StreamingBody object at 0x1117e0f70>"
}


In [24]:
logger.info(response)

INFO: 2025-08-22 11:43:29 [3676231549.py:1] {
    "ResponseMetadata": {
        "RequestId": "957bd7e4-c65c-4108-8db6-7da9643a5f9c",
        "HTTPStatusCode": 202,
        "HTTPHeaders": {
            "date": "Fri, 22 Aug 2025 16:43:25 GMT",
            "content-length": "0",
            "connection": "keep-alive",
            "x-amzn-requestid": "957bd7e4-c65c-4108-8db6-7da9643a5f9c",
            "x-amzn-remapped-content-length": "0",
            "x-amzn-trace-id": "root=1-68a89e2d-318a7aff5df07edf6e9317e0;parent=29880cda2983f79f;sampled=0"
        },
        "RetryAttempts": 0
    },
    "StatusCode": 202,
    "Payload": "<botocore.response.StreamingBody object at 0x111ef9de0>"
}


In [ ]:

lfn_client.update_function_configuration(
    FunctionName=LFN_NAME,
    Timeout=120,  # in seconds
)


In [22]:
# Read the response
# response_payload = json.loads(response["Payload"].read())
response_payload = response["Payload"].read().decode("utf-8")
print(response_payload)

##### Extract Lambda logs

In [56]:
log_manager = LogManager(region_name="us-east-1")
lambda_extractor = log_manager.get_lambda_extractor()

# end_time = datetime.now(timezone.utc)
end_time = datetime.now()
start_time = end_time - timedelta(hours=8)
function_name = "sqs-processor"
function_name = LFN_NAME
request_id = response["ResponseMetadata"]["RequestId"]

# lambda_logs = lambda_extractor.get_logs_by_time_range(
#     function_name=function_name,
#     start_time=start_time,
#     end_time=end_time,
#     # log_filter=LogFilter(filter_pattern="ERROR"),
#     log_filter=LogFilter(),
# )

lambda_logs = lambda_extractor.get_logs_by_strem_limit(function_name=function_name)

# lambda_logs = lambda_extractor.get_logs_by_request_id(function_name, request_id)

print(lambda_logs)

# Print results
for log in lambda_logs:
    # print(f"{log.timestamp}: {log.level} - {log.message}")
    print(log.message)


[LogEntry(timestamp=datetime.datetime(2025, 8, 23, 2, 24, 2, 102000, tzinfo=datetime.timezone.utc), message='INIT_START Runtime Version: python:3.9.v101\tRuntime Version ARN: arn:aws:lambda:us-east-1::runtime:af29e10439856e364100c5dec1ce9c55d44feb2772258f7a7e480a95474aa18f\n', level=None, request_id=None, source='LambdaLogExtractor', metadata={'ingestionTime': 1755915851161}), LogEntry(timestamp=datetime.datetime(2025, 8, 23, 2, 24, 3, 139000, tzinfo=datetime.timezone.utc), message='START RequestId: 87c27f75-91f5-4630-bc1d-44890bf26a35 Version: $LATEST\n', level=None, request_id=None, source='LambdaLogExtractor', metadata={'ingestionTime': 1755915851161}), LogEntry(timestamp=datetime.datetime(2025, 8, 23, 2, 24, 4, 662000, tzinfo=datetime.timezone.utc), message='INFO: 2025-08-23 02:24:04 [lambda_handler.py:144] Event sent to lambda_handler:\n', level='INFO', request_id=None, source='LambdaLogExtractor', metadata={'ingestionTime': 1755915851161}), LogEntry(timestamp=datetime.datetime(20

#### Trigger CloudWatch Event

In [ ]:
# List all rules associated with the given prefix
rules = events_client.list_rules(NamePrefix="come_to_work")["Rules"]
rules = events_client.list_rules(NamePrefix="daily_tasks")["Rules"]
rules = events_client.list_rules(NamePrefix="pickup")["Rules"]
print(rules)

[{'Name': 'come_to_work', 'Arn': 'arn:aws:events:us-east-1:530976901147:rule/come_to_work', 'State': 'ENABLED', 'ScheduleExpression': 'cron(0 12 ? * MON-FRI *)', 'EventBusName': 'default'}]


In [70]:
response = events_client.describe_rule(Name="come_to_work", EventBusName="default")
print(response)

{'Name': 'come_to_work', 'Arn': 'arn:aws:events:us-east-1:530976901147:rule/come_to_work', 'ScheduleExpression': 'cron(0 12 ? * MON-FRI *)', 'State': 'ENABLED', 'EventBusName': 'default', 'CreatedBy': '530976901147', 'ResponseMetadata': {'RequestId': '992a8a6e-b7ba-40d9-8524-5029ae4df544', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '992a8a6e-b7ba-40d9-8524-5029ae4df544', 'content-type': 'application/x-amz-json-1.1', 'content-length': '205', 'date': 'Sat, 23 Aug 2025 23:06:28 GMT'}, 'RetryAttempts': 0}}


In [ ]:
events_client.disable_rule(Name="come_to_work", EventBusName="default")

In [ ]:
events_client.enable_rule(Name="come_to_work", EventBusName="default")

In [4]:
# Update (or create) a rule
response = events_client.put_rule(
    Name="come_to_work",  # Rule name
    ScheduleExpression="cron(41 23 ? * MON-SUN *)",  # Update schedule (can be cron or rate)
    # ScheduleExpression="rate(2 minutes)",  # Update schedule (can be cron or rate)
    State="ENABLED",  # ENABLED | DISABLED
    Description="Updated schedule for my Lambda trigger",
    # RoleArn="arn:aws:iam::123456789012:role/EventBridgeRole",  # optional, only for cross-account or certain targets
)


In [72]:
print(response)

{'RuleArn': 'arn:aws:events:us-east-1:530976901147:rule/come_to_work', 'ResponseMetadata': {'RequestId': 'abecc332-fb9a-4fcf-83d0-7ea0dd506719', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'abecc332-fb9a-4fcf-83d0-7ea0dd506719', 'content-type': 'application/x-amz-json-1.1', 'content-length': '69', 'date': 'Sat, 23 Aug 2025 23:28:30 GMT'}, 'RetryAttempts': 0}}


#### Publish Messages into SNS

In [ ]:
# Create SNS client
sns = boto3.client("sns")

# Your SNS Topic ARN
topic_arn = "arn:aws:sns:us-east-1:530976901147:lambda-success-topic"

# Message payload
message = {
    "event": "LambdaExecutionSuccess",
    "detail": {
        "function": "my-lambda-function",
        "status": "SUCCESS",
        "timestamp": "2025-08-18T15:30:00Z",
    },
}

# Publish message
response = sns.publish(
    TopicArn=topic_arn,
    Message=json.dumps(message),  # Body of the message
    Subject="Lambda Success Notification",  # Optional
)

print("MessageId:", response["MessageId"])


#### Send Messages into SQS

In [ ]:
# Read Input Queue URL from env var or hardcode it
INPUT_QUEUE_URL = os.getenv(
    "INPUT_QUEUE_URL",
    "https://sqs.us-east-1.amazonaws.com/530976901147/lambda-input-queue",
)

# Initialize boto3 SQS client
sqs_client = boto3.client("sqs")


def send_message(message_body: dict):
    """
    Publish a single message to SQS
    """
    response = sqs_client.send_message(
        QueueUrl=INPUT_QUEUE_URL, MessageBody=json.dumps(message_body)
    )
    print(f"Message sent: {response['MessageId']}")
    return response


# Example test payload
test_payload = {
    "eventType": "TestEvent",
    "timestamp": datetime.now().isoformat(),
    "data": {
        "userId": "12345",
        "action": "process_data",
        "payload": "Hello from SQS test",
    },
}

send_message(test_payload)

# Optionally, send multiple test messages
for i in range(1):
    send_message({"eventType": "BatchTest", "index": i, "payload": f"Test message {i}"})


#### Reserved & Provisioned Concurrency

-   AWS always reserves 100 concurrency per Region for all functions combined, unless you’ve asked AWS Support to lower it.
-   You can assign reserved concurrency to individual functions, but AWS requires you to leave at least 10 unreserved concurrency available at the account level.
-   Your reserved_concurrent_executions = 3 on aws_lambda_function.gpc_cuckoo is fine in isolation — but combined with the 11 provisioned concurrency you set on the alias, Terraform is effectively trying to over-allocate concurrency.
-   

-   🔑**Key Points**

    -   Reserved concurrency (reserved_concurrent_executions) = hard cap per function. Prevents “runaway” invocations.
    -   Provisioned concurrency = pre-warmed execution environments. But it counts against your function’s reserved concurrency.
    -   If you configure provisioned_concurrent_executions = 11 but only allow reserved_concurrent_executions = 3, it’s invalid. Provisioned concurrency must be ≤ reserved concurrency (or unreserved account pool).

#### Security Group Confussion

Good question 👍 — this often confuses people when wiring **Lambda ↔ EFS** inside a VPC.

Let’s walk through it:

1. How Security Groups Work in AWS

    * Security groups are **stateful** firewalls.
    * If **inbound** traffic is allowed, the **response** traffic is automatically allowed back (no need to open outbound explicitly for that flow).
    * BUT security groups are **attached to each resource independently** — i.e., Lambda has its own SG, EFS mount target has its own SG.

    So:

    * Lambda SG controls **what Lambda can send out to**.
    * EFS SG controls **what EFS can accept inbound from**.

2. Your Case (Lambda → EFS over NFS/port 2049)

    * **Lambda → EFS (outbound from Lambda SG)**:
      Lambda needs permission to initiate the TCP connection **outbound** to port 2049 on EFS.
      → That’s why you created **`lambda_to_efs`**.

    * **EFS ← Lambda (inbound to EFS SG)**:
      The EFS mount target needs to allow inbound connections on port 2049 **from Lambda SG**.
      → That’s why you created **`efs_from_lambda`**.

    Both rules are required because:

    * Lambda SG doesn’t know about EFS unless explicitly allowed.
    * EFS SG doesn’t know about Lambda unless explicitly allowed.
    * Security groups don’t “merge permissions”; each resource enforces its own SG rules.

3. Why Can’t I Just Create One Rule?

    Because each security group applies only to the resource it’s attached to:

    * If you only configure **inbound on EFS SG**, Lambda’s SG still won’t allow initiating the connection.
    * If you only configure **outbound on Lambda SG**, EFS SG will block it because it doesn’t allow traffic from Lambda’s SG.

    So both sides must explicitly allow each other.


-   **Summary**:
    You need both `lambda_to_efs` and `efs_from_lambda` because:

    * **Lambda SG** must allow outbound traffic to EFS.
    * **EFS SG** must allow inbound traffic from Lambda.

    That’s the standard AWS pattern for **one service talking to another inside a VPC** when each has its own SG.
